In [65]:
%load_ext autoreload
%autoreload 2

import glob
import numpy as np
import os
import shutil

from torchvision.io import read_image, write_png
from tqdm import tqdm

from src.deep_ad.config import Config
from src.deep_ad.data.dagm_dataset import DAGMDatasetDev
from src.deep_ad.image import intersection_over_union, TBBox, show_image_with_bboxes, show_image

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
# Load the configuration
config = Config(root_dir="..")

The reconstruction CNN uses only defect-free patches so I need to prepare a dataset containing them. I will take a 
constant number of patches from each image from the raw dataset. According to the original paper, they cropped patches
larger than 128x128 pixels in order to avoid border effects after applying random transforms. However, they didn't
specify the size of the patches.

For better training, I thought that patches taken from the same image need to cover as much details as possible, so 
overlappings should be minimized. Thus, the intersection over union between each pair of patches must not exceed a
certain threshold.

Patches obtained will be saved in a folder structure identical to that of the raw DAGM dataset. However, each patch will
contain the name, id (integer from `0` to `patches_per_image - 1`) and the `x` and `y` coordinates of the top-left 
corner. This new dataset will be saved in a directory uniquely identified by its configuration parameters: ppi (patches
per image) and patch size in pixels.

Existing patches should be deleted, as new ones might not overwrite them.

In [105]:
patches_dir = os.path.join(config.DAGM_processed_dir, f"{config.patches_per_image}_ppi_{config.raw_patch_size}px")
if not os.path.exists(patches_dir):
    print(f"Directory \"{patches_dir}\" does not exist. Exiting.")
else:
    number_of_files = len(glob.glob(os.path.join(patches_dir, "Class*", "Train", "*.png")))
    prompt = f"Are you sure you want to delete {number_of_files} files inside \"{patches_dir}\"? (y/n) "
    response = input(prompt).strip().lower()
    if response != "y":
        print("Exiting.")
    else:
        print(
            f"Deleting {number_of_files} files inside \"{patches_dir}\"..."
        )
        shutil.rmtree(patches_dir)
        print("Done.")

Directory "..\data\processed\DAGM\4_ppi_160px" does not exist. Exiting.


Load only defect-free images and crop a fixed number of patches from each one of them.

In [98]:
dagm_ds = DAGMDatasetDev(config.DAGM_raw_dir, type="Defect-free")

In [106]:
patches_dir = os.path.join(config.DAGM_processed_dir, f"{config.patches_per_image}_ppi_{config.raw_patch_size}px")
print("Config:")
print(f"\tpatches_per_image: {config.patches_per_image}")
print(f"\traw_patch_size: {config.raw_patch_size}")
print(f"\tpatches_iou_threshold: {config.patches_iou_threshold}")
print(f"Will save patches to \"{patches_dir}\".")

# Create directories for each class
for cls in dagm_ds.all_classes:
    cls_patches_dir = os.path.join(patches_dir, f"Class{cls}", "Train")
    if not os.path.exists(cls_patches_dir):
        os.makedirs(cls_patches_dir)

# Generate patches
for i in tqdm(range(len(dagm_ds))):
    image, label, cls, name = dagm_ds[i]
    cls_patches_dir = os.path.join(patches_dir, f"Class{cls}", "Train")
    patch_bboxes: list[TBBox] = []

    for _ in range(config.patches_per_image):
        # Keep generating random coordinates while iou is greater than threshold
        while True:
            top_left_x = np.random.randint(0, image.shape[1] - config.raw_patch_size + 1)
            top_left_y = np.random.randint(0, image.shape[0] - config.raw_patch_size + 1)
            bbox: TBBox = (
                top_left_x,
                top_left_y,
                top_left_x + config.raw_patch_size - 1,
                top_left_y + config.raw_patch_size - 1,
            )
            if np.all(
                [intersection_over_union(bbox, bbox_) < config.patches_iou_threshold for bbox_ in patch_bboxes]
            ):
                patch_bboxes.append(bbox)
                break

    # Save the patches
    for i, bbox in enumerate(patch_bboxes):
        patch = image[bbox[1] : bbox[3] + 1, bbox[0] : bbox[2] + 1].unsqueeze(0)
        patch_name = f"{name}_{i}_{bbox[0]}_{bbox[1]}.png"
        patch_path = os.path.join(cls_patches_dir, patch_name)
        write_png(patch, patch_path)

print(f"Generated a total of {len(glob.glob(os.path.join(patches_dir, 'Class*', 'Train', '*.png')))} patches.")

Config:
	patches_per_image: 4
	raw_patch_size: 160
	patches_iou_threshold: 0.5
Will save patches to "..\data\processed\DAGM\4_ppi_160px".


100%|██████████| 7004/7004 [01:14<00:00, 94.61it/s] 


Generated a total of 28016 patches.


Display an image and its bounding boxes.

In [108]:
image_cls = 2
image_name = "0577"
patches_dir = os.path.join(
    config.DAGM_processed_dir, f"{config.patches_per_image}_ppi_{config.raw_patch_size}px", f"Class{image_cls}", "Train"
)
bboxes: list[TBBox] = []
for i in range(config.patches_per_image):
    patch_path = os.path.join(patches_dir, f"{image_name}_{i}_*.png")
    patch_path = glob.glob(patch_path)[0]
    tl_x = int(patch_path.split("_")[-2])
    tl_y = int(patch_path.split("_")[-1].split(".")[0])
    bboxes.append((tl_x, tl_y, tl_x + config.raw_patch_size - 1, tl_y + config.raw_patch_size - 1))

image = (
    read_image(os.path.join(config.DAGM_raw_dir, f"Class{image_cls}", "Train", f"{image_name}.png")).squeeze(0).numpy()
)
show_image_with_bboxes(image, bboxes)